In [ ]:
# --------------------------------------------------#
# 0. Uploading Artur's prediction CSV file directly #
# --------------------------------------------------#

from google.colab import files
uploaded = files.upload()


df_pred_up_ah = list(uploaded.keys())[0]


In [ ]:
### artur's block

#-------------------------------------------------------#
# Creating sepparate results based on depth and network #
#-------------------------------------------------------#

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

# -----------------------------
# 1. Feature engineering
# -----------------------------
def add_features(group):
    group = group.copy()
    # Rolling and lagged climate features
    group["precip_roll3"] = group["precipitation"].rolling(3, min_periods=1).mean()
    group["temp_roll3"] = group["temperature"].rolling(3, min_periods=1).mean()
    group["precip_lag1"] = group["precipitation"].shift(1)
    group["precip_sum7"] = group["precipitation"].rolling(7, min_periods=1).sum()
    group["precip_sum14"] = group["precipitation"].rolling(14, min_periods=1).sum()
    group["temp_lag1"] = group["temperature"].shift(1)
    group["temp_mean7"] = group["temperature"].rolling(7, min_periods=1).mean()
    group["temp_mean30"] = group["temperature"].rolling(30, min_periods=1).mean()
    # Lagged soil moisture
    group["sm_lag1"] = group["soil_moisture"].shift(1)
    group["sm_lag3"] = group["soil_moisture"].rolling(3, min_periods=1).mean()
    # Water balance
    group["water_balance"] = group["precip_sum7"] - group["temp_mean7"]
    # Seasonality
    group["day_of_year"] = group["date"].dt.dayofyear
    group["month"] = group["date"].dt.month
    return group

# -----------------------------
# 2. Training function
# -----------------------------
def train_rf_models(df):
    results_depth = {}
    results_network = {}

    feature_cols = [
        "precip_roll3", "temp_roll3", "precip_lag1", "precip_sum7", "precip_sum14",
        "temp_lag1", "temp_mean7", "temp_mean30", "sm_lag1", "sm_lag3",
        "water_balance", "day_of_year", "month"
    ]

    # Group by depth
    for depth, group in df.groupby("sm_depth_from"):
        X = group[feature_cols]
        y = group["soil_moisture"]
        mask = X.notna().all(axis=1)
        X, y = X[mask], y[mask]

        #split data into training and test folds while respecting temporal order
        tscv = TimeSeriesSplit(n_splits=5)
        scores, rmses = [], []
        for train_idx, test_idx in tscv.split(X):
            #split data retaining chronological order
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            model = RandomForestRegressor(
                n_estimators=200, max_depth=None, min_samples_leaf=5,
                max_features="sqrt", random_state=42
            )
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            scores.append(r2_score(y_test, y_pred))
            rmses.append(np.sqrt(mean_squared_error(y_test, y_pred)))

        results_depth[depth] = {
            "mean_r2": np.mean(scores),
            "mean_rmse": np.mean(rmses),
            "last_model": model
        }

    # Group by network
    for network, group in df.groupby("network"):
        X = group[feature_cols]
        y = group["soil_moisture"]
        mask = X.notna().all(axis=1)
        X, y = X[mask], y[mask]


        #split data into training and test folds while respecting temporal order
        tscv = TimeSeriesSplit(n_splits=5)
        scores, rmses = [], []
        for train_idx, test_idx in tscv.split(X):
            #split data retaining chronological order
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            model = RandomForestRegressor(
                n_estimators=200, max_depth=None, min_samples_leaf=5,
                max_features="sqrt", random_state=42
            )
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            scores.append(r2_score(y_test, y_pred))
            rmses.append(np.sqrt(mean_squared_error(y_test, y_pred)))

        results_network[network] = {
            "mean_r2": np.mean(scores),
            "mean_rmse": np.mean(rmses),
            "last_model": model
        }

    return results_network, results_depth

# ------------------#
# 3. Apply training #
# ------------------#

df_ah = pd.read_csv(
    filename_ah,
    parse_dates=["date"]
)
df_ah = df_ah.sort_values(["station", "sm_depth_from", "date"])
df_ah = df_ah.groupby(["network", "sm_depth_from"]).apply(add_features).reset_index(drop=True)
df_ah = df_ah.dropna(subset=["soil_moisture", "precipitation", "temperature"])

results_network, results_depth = train_rf_models(df_ah)

print("Network-level results:")
for net, res in results_network.items():
    print(f"{net}: R2={res['mean_r2']:.3f}, RMSE={res['mean_rmse']:.3f}")

print("\nDepth-level results:")
for depth, res in results_depth.items():
    print(f"{depth}m: R2={res['mean_r2']:.3f}, RMSE={res['mean_rmse']:.3f}")




In [ ]:

#----------------------------------------#
# Artur's RF predicting on distinct file #
#----------------------------------------#

# -----------------------------
# 4. Predict on new data
# -----------------------------
df_pred_ah = pd.read_csv(
    df_pred_up_ah,
    parse_dates=["date"]
)
df_pred_ah = df_pred_ah.sort_values(["station", "sm_depth_from", "date"])
df_pred_ah = df_pred_ah.groupby(["network", "sm_depth_from"]).apply(add_features).reset_index(drop=True)
df_pred_ah = df_pred_ah.dropna(subset=["precip_roll3", "temp_roll3"])

feature_cols = [
    "precip_roll3", "temp_roll3", "precip_lag1", "precip_sum7", "precip_sum14",
    "temp_lag1", "temp_mean7", "temp_mean30", "sm_lag1", "sm_lag3",
    "water_balance", "day_of_year", "month"
]

predictions_net, predictions_depth = [], []

for depth, group in df_pred_ah.groupby("sm_depth_from"):
    if depth in results_depth:
        model = results_depth[depth]["last_model"]
        X_pred = group[feature_cols]
        group["soil_moisture_pred"] = model.predict(X_pred)
        predictions_depth.append(group)

for network, group in df_pred_ah.groupby("network"):
    if network in results_network:
        model = results_network[network]["last_model"]
        X_pred = group[feature_cols]
        group["soil_moisture_pred"] = model.predict(X_pred)
        predictions_net.append(group)

df_pred_net = pd.concat(predictions_net)
df_pred_depth = pd.concat(predictions_depth)

df_pred_net[["date", "station", "soil_moisture_pred", "precipitation", "temperature"]].to_csv(
    "predicted_soilmoisture_network.csv", index=False)
df_pred_depth[["date", "station", "soil_moisture_pred", "precipitation", "temperature"]].to_csv(
    "predicted_soilmoisture_depth.csv", index=False)



In [ ]:
#------------------------#
# Artur's RF RMSE values #
#------------------------#


rmse_values = [metrics["mean_rmse"] for metrics in results_depth.values()]
mean_rmse = np.mean(rmse_values)

print("Mean RMSE across all station-depths based on depth model:", mean_rmse)

rmse_values = [metrics["mean_rmse"] for metrics in results_network.values()]
mean_rmse = np.mean(rmse_values)

print("Mean RMSE across all station-depths based on network model:", mean_rmse)




In [ ]:
#----------------------------#
# Artur's RF results ploting #
#----------------------------#

import pandas as pd
import matplotlib.pyplot as plt


df_real = pd.read_csv(df_pred_up_ah, parse_dates=["date"])
df_real = df_real.sort_values(["station", "sm_depth_from", "date"])

# Choose one station and year for clarity
station_name = "MoorHouse"
year = 2017

df_real_sub = df_real[
    (df_real["station"] == station_name) &
    (df_real["date"].dt.year == year)
    ]
df_net_sub = df_pred_net[(df_pred_net["station"] == station_name) & (df_pred_net["date"].dt.year == year)]
df_depth_sub = df_pred_depth[(df_pred_depth["station"] == station_name) & (df_pred_depth["date"].dt.year == year)]

# Plot
fig, ax = plt.subplots(3, 1, figsize=(16, 15))


# Predictions from network model
ax[0].plot(df_net_sub["date"], df_net_sub["soil_moisture_pred"],
           label="Ennustatud mullaniiskus (network)", color="green", alpha=0.3)

# Predictions from depth model
ax[0].plot(df_depth_sub["date"], df_depth_sub["soil_moisture_pred"],
           label="Ennustatud mullaniiskus (depth)", color="orange", alpha=0.3)

ax[0].set_ylabel("Predicted soil moisture")
ax[0].legend(loc="upper right")

# Real soil moisture by depth
colors = {0.1: "red", 0.3: "blue", 0.6: "darkblue"}  # assign colors per depth

for depth, group in df_real_sub.groupby("sm_depth_to"):
    ax[1].plot(group["date"], group["soil_moisture"],
               label=f"Tegelik mullaniiskus sügavusel kuni {depth} m",
               color=colors.get(depth, "black"), linestyle="--")

ax[1].set_ylabel("Real soil moisture")
ax[1].legend(loc="upper right")

ax[1].set_ylabel("Real soil moisture")
ax[1].legend(loc="upper right")

# Climate drivers subplot
ax[2].plot(df_real_sub["date"], df_real_sub["precipitation"],
           label="Sademed", color="blue", alpha=0.5)
'''ax[1].plot(df_real_sub["date"], df_real_sub["temperature"],
           label="Temperatuur", color="red", alpha=0.7)'''

ax[2].set_ylabel("Climate drivers")
ax[2].legend(loc="upper right")

fig.suptitle(f"Mõõdetud ja ennustatud mullaniiskus ({station_name}, {year})")
plt.show()


In [ ]:
#----------------------------#
# Artur's RF results ploting #
#----------------------------#

import os
import pandas as pd
import matplotlib.pyplot as plt

folder = "/folder/path/of/csv"
files = [f for f in os.listdir(folder) if f.endswith(".csv")]


fig, axes = plt.subplots(8, 3, figsize=(40, 60))
axes = axes.flatten()


for i, file in enumerate(files):
    df = pd.read_csv(os.path.join(folder, file))


    df['date'] = pd.to_datetime(df['date'])
    df_2024 = df[df['date'].dt.year == 2024]
    name = file.split("_")
    axes[i].plot(df_2024['date'], df_2024['predicted_soil_moisture'])
    axes[i].set_title(name[0], fontsize=30, fontweight="bold")
    axes[i].tick_params(axis='x', labelsize=25, labelrotation=45)
    axes[i].tick_params(axis='y', labelsize=25, labelrotation=45)

fig.suptitle("Estimated soilmoisture of Estonian weatherstations 2024", fontsize=40, fontweight="bold")
fig.supylabel("Soil Moisture (m3*m3)", fontsize=40, fontweight="bold")
plt.tight_layout(rect=[0.02, 0, 1, 0.98])
plt.savefig("/path/to/export/pdf/soilmoisture_2024_plot.pdf", bbox_inches="tight")
plt.show()




